# Analysis and Prediction of Heart Disease
### (Feature Creation)
## 3. Feature Creation :

In [2]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3.eu-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-39b8034e-49e4-4e3e-b6fd-44f01e7f35b9',
    'iam_service_endpoint': 'https://iam.eu-gb.bluemix.net/oidc/token',
    'api_key': 'iN3J17eRe8Mezyc57J4mtxTRFl-oey7Re0eCAwWgP_8T'
}

configuration_name = 'os_0dc47b3a85024faf8c9e361e756597e5_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')


from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_data = spark.read.parquet(cos.url('heart.parquet', 'definition-donotdelete-pr-cluwtgruaxvlbz'))
print("Number of records = ", df_data.count(), "\n")
df_data.createOrReplaceTempView('df')
df_data.show()

Number of records =  303 

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|
| 37|  1|  2|     130| 250|  0|      1|    187|    0|    3.5|    0|  0|   2|     1|
| 41|  0|  1|     130| 204|  0|      0|    172|    0|    1.4|    2|  0|   2|     1|
| 56|  1|  1|     120| 236|  0|      1|    178|    0|    0.8|    2|  0|   2|     1|
| 57|  0|  0|     120| 354|  0|      1|    163|    1|    0.6|    2|  0|   2|     1|
| 57|  1|  0|     140| 192|  0|      1|    148|    0|    0.4|    1|  0|   1|     1|
| 56|  0|  1|     140| 294|  0|      0|    153|    0|    1.3|    1|  0|   2|     1|
| 44|  1|  1|     120| 263|  0|      1|    173|    0|    0.0|    2|  0|   3|     1|
| 52|  1|  2|     172| 199|  1|      1|    162|  

### 3.2. Feature Engineering :

Let's apply One Hot Encoding to the categorical integer features.These are sex, cp, fbs, restecg, exang, slope, ca ve thal.

In [11]:
from pyspark.ml.feature import OneHotEncoder

# Create one hot encoders for the categorical features
encoder1 = OneHotEncoder(inputCol = 'sex', outputCol = 'sexVec')
encoder2 = OneHotEncoder(inputCol = 'cp', outputCol = 'cpVec')
encoder3 = OneHotEncoder(inputCol = 'fbs', outputCol = 'fbsVec')
encoder4 = OneHotEncoder(inputCol = 'restecg', outputCol = 'restecgVec')
encoder5 = OneHotEncoder(inputCol = 'exang', outputCol = 'exangVec')
encoder6 = OneHotEncoder(inputCol = 'slope', outputCol = 'slopeVec')
encoder7 = OneHotEncoder(inputCol = 'ca', outputCol = 'caVec')
encoder8 = OneHotEncoder(inputCol = 'thal', outputCol = 'thalVec')

Merge the features into a single features vector and then normalize them.

In [12]:
from pyspark.ml.feature import MinMaxScaler, VectorAssembler

# Create a features vector
vectorAssembler = VectorAssembler(inputCols = ['sexVec', 'cpVec', 'trestbps', 'chol', 'fbsVec', 'restecgVec', 
                                               'thalach', 'exangVec', 'oldpeak', 'slopeVec', 'caVec','thalVec'],
                                  outputCol = 'featuresVec')

# Normalize the features data
normalizer = MinMaxScaler(inputCol = 'featuresVec', outputCol = 'features')

In [13]:
from pyspark.ml import Pipeline

# Create a Feature Engineering ML pipeline
pipeline = Pipeline(stages = [encoder1, encoder2, encoder3, encoder4, encoder5, encoder6, encoder7, encoder8, vectorAssembler, normalizer])
df_normalized_data = pipeline.fit(df_data).transform(df_data)
df_normalized_data.show()

+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+--------------------+
|age|sex| cp|trestbps|chol|fbs|restecg|thalach|exang|oldpeak|slope| ca|thal|target|       sexVec|        cpVec|       fbsVec|   restecgVec|     exangVec|     slopeVec|        caVec|      thalVec|         featuresVec|            features|
+---+---+---+--------+----+---+-------+-------+-----+-------+-----+---+----+------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+--------------------+--------------------+
| 63|  1|  3|     145| 233|  1|      0|    150|    0|    2.3|    0|  0|   1|     1|    (1,[],[])|    (3,[],[])|    (1,[],[])|(2,[0],[1.0])|(1,[0],[1.0])|(2,[0],[1.0])|(4,[0],[1.0])|(3,[1],[1.0])|(21,[4,5,7,9,10,1...|[0.0,0.0,0.0,0.0,...|
| 37|  1|  2|     130| 250|  0|      1|    187| 

We have created the normalized features, let's drop all the other columns from our dataset and features vector except target and features.

In [18]:
df_normalized_data = df_normalized_data.drop('age').drop('sex').drop('cp').drop('trestbps').drop('chol') \
                        .drop('fbs').drop('restecg').drop('thalach').drop('exang').drop('oldpeak').drop('slope') \
                        .drop('ca').drop('thal').drop('sexVec').drop('cpVec').drop('fbsVec').drop('restecgVec') \
                        .drop('exangVec').drop('slopeVec').drop('caVec').drop('thalVec').drop('featuresVec')
                        
df_normalized_data.show()

+------+--------------------+
|target|            features|
+------+--------------------+
|     1|[0.0,0.0,0.0,0.0,...|
|     1|[0.0,0.0,0.0,1.0,...|
|     1|[1.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,1.0,0.0,...|
|     1|[1.0,1.0,0.0,0.0,...|
|     1|[0.0,1.0,0.0,0.0,...|
|     1|[1.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,0.0,1.0,...|
|     1|[0.0,0.0,0.0,1.0,...|
|     1|[0.0,1.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,1.0,...|
|     1|[0.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,1.0,...|
|     1|[1.0,0.0,0.0,1.0,...|
|     1|[1.0,0.0,0.0,0.0,...|
|     1|[0.0,1.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,0.0,...|
+------+--------------------+
only showing top 20 rows



### 3.3. Store Feature Engineered Data in IBM Object Store :

We will use this data set(heart_normalized.parquet) our future modeling, prediction and evaluation.

In [19]:
df_normalized_data = df_normalized_data.repartition(1)
df_normalized_data.write.parquet(cos.url('heart_normalized.parquet','definition-donotdelete-pr-cluwtgruaxvlbz'))

In [20]:
df_persisted_data = spark.read.parquet(cos.url('heart_normalized.parquet', 
                                               'definition-donotdelete-pr-cluwtgruaxvlbz'))
print('Number of records persisted = ', df_persisted_data.count())
df_persisted_data.show()

Number of records persisted =  303
+------+--------------------+
|target|            features|
+------+--------------------+
|     1|[0.0,0.0,0.0,0.0,...|
|     1|[0.0,0.0,0.0,1.0,...|
|     1|[1.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,1.0,0.0,...|
|     1|[1.0,1.0,0.0,0.0,...|
|     1|[0.0,1.0,0.0,0.0,...|
|     1|[1.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,0.0,1.0,...|
|     1|[0.0,0.0,0.0,1.0,...|
|     1|[0.0,1.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,1.0,...|
|     1|[0.0,0.0,1.0,0.0,...|
|     1|[0.0,0.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,1.0,...|
|     1|[1.0,0.0,0.0,1.0,...|
|     1|[1.0,0.0,0.0,0.0,...|
|     1|[0.0,1.0,0.0,0.0,...|
|     1|[1.0,0.0,0.0,0.0,...|
+------+--------------------+
only showing top 20 rows



Our normalized data set has been successfully written to IBM object storage and, there is not any data loss.